In [42]:
#Question 3
import numpy as np
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable
model = LpProblem(name='DSHW1Q5', sense=LpMaximize)
x = LpVariable(name="x")
y = LpVariable(name="y")
#z = LpVariable(name="z")
model += (x + 2*y <= 14, "constraint_1")
model += (3*x - y >= 0, "constraint_2")
model += (x - y <= 2, "constraint_3")
model += 5*x + 3*y
model
status = model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: 5x +3y = {model.objective.value()}")
for var in model.variables():
    print(f"{var.name}: {var.value()}")

status: 1, Optimal
objective: 5x +3y = 42.0
x: 6.0
y: 4.0


In [29]:
#Question 5
import pandas as pd
import networkx as nx

# Step 1: Load the Data
edges_path = r'C:\Users\16366\Downloads\lastfm_asia\lasftm_asia\lastfm_asia_edges.csv'
targets_path = r'C:\Users\16366\Downloads\lastfm_asia\lasftm_asia\lastfm_asia_target.csv'

edges_df = pd.read_csv(edges_path)
target_df = pd.read_csv(targets_path)

# Step 2: Create the Graph
G = nx.Graph()

# Step 3: Add Nodes and Edges
G.add_edges_from(zip(edges_df['node_1'], edges_df['node_2']))

# Step 4: Associate Targets with Nodes
target_mapping = dict(zip(target_df['id'], target_df['target']))
nx.set_node_attributes(G, target_mapping, 'target')

# Step 5: Calculate Average Shortest Path Length from the first node
start_node = list(G.nodes())[0]
total_path_length = 0
num_nodes = len(G.nodes()) - 1

for node in G.nodes():
    if node != start_node:
        shortest_path_length = nx.shortest_path_length(G, start_node, node)
        total_path_length += shortest_path_length

average_shortest_path_length = total_path_length / num_nodes

print(f"The average shortest path length from the first node is: {average_shortest_path_length:.3f}")

The average shortest path length from the first node is: 5.652


In [30]:
#Question 6
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests
import matplotlib.pyplot as plt
from datetime import date,datetime

url = 'https://www.worldometers.info/coronavirus/#countries'
req = requests.get(url)
page_soup = soup(req.text, "html.parser")
current_datetime = datetime.now()
formatted_datetime = current_datetime.strftime('%Y-%m-%d %H:%M:%S')
print("Data from:",formatted_datetime)

table = page_soup.findAll("table", {"id": "main_table_countries_today"})

containers = table[0].findAll("tr") 

del containers[0] 

all_data = [] 
for country in containers: 
    country_data = []
    country_container = country.findAll("td") 
 
    for i in range(1, len(country_container)):
        final_feature = country_container[i].text
        clean=1
        
        if clean:
            if i != 1 and i != len(country_container)-1:
                final_feature = final_feature.replace(',', '')
                if final_feature.find('+') != -1:
                    final_feature = final_feature.replace('+', '')
                    final_feature = float(final_feature)
                elif final_feature.find('-') != -1:
                    final_feature = final_feature.replace('-', '')
                    final_feature = float(final_feature)*-1

        
        if final_feature == 'N/A':
            final_feature = 0
        elif final_feature == '' or final_feature == ' ':
            final_feature = 0 
        country_data.append(final_feature)
    all_data.append(country_data)
df = pd.DataFrame(all_data)
df = df.drop([i for i in range(15, len(all_data[0]))], axis=1) 
column_labels = ["Country", "Total Cases", "New Cases", "Total Deaths", "New Deaths", "Total Recovered", "New Recovered", "Active Cases", "Serious/Critical",
                "Tot Cases/1M", "Deaths/1M", "Total Tests", "Tests/1M", "Population", "Continent"]
df.columns = column_labels
df


Data from: 2023-11-02 14:52:12


,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious/Critical,Tot Cases/1M,Deaths/1M,Total Tests,Tests/1M,Population,Continent
0,\nNorth America\n,128997336,0.0,1652633,0.0,124719053,1334.0,2625650,6549,0,0,0,0,0,North America
1,\nAsia\n,220810861,82.0,1551534,2.0,204530282,17.0,14729045,15143,0,0,0,0,0,Asia
2,\nEurope\n,250790529,1039.0,2078038,2.0,246633821,3506.0,2078670,5649,0,0,0,0,0,Europe
3,\nSouth America\n,69178247,0.0,1362310,0.0,66556462,0.0,1259475,10098,0,0,0,0,0,South America
4,\nOceania\n,14698883,0.0,30699,0.0,14561851,0.0,106333,49,0,0,0,0,0,Australia/Oceania
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,Total:,69178247,0.0,1362310,0.0,66556462,0.0,1259475,10098,0,0,0,0,0,South America
243,Total:,14698883,0.0,30699,0.0,14561851,0.0,106333,49,0,0,0,0,0,Australia/Oceania
244,Total:,12843500,0.0,258843,0.0,12089243,0.0,495414,547,0,0,0,0,0,Africa
245,Total:,721,0.0,15,0.0,706,0.0,0,0,0,0,0,0,0,0


In [31]:
country_labels = page_soup.findAll("a", {"class": "mt_a"})

c_label = []
for country in country_labels:
    c_label.append(country.text)
c_label = set(c_label)

not_counted = []
sorted_countries = set(df['Country']) 
for country in c_label:
    if country not in sorted_countries:
        not_counted.append(country)

print(not_counted)

[]


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247 entries, 0 to 246
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country           247 non-null    object 
 1   Total Cases       247 non-null    object 
 2   New Cases         247 non-null    float64
 3   Total Deaths      247 non-null    object 
 4   New Deaths        247 non-null    float64
 5   Total Recovered   247 non-null    object 
 6   New Recovered     247 non-null    float64
 7   Active Cases      247 non-null    object 
 8   Serious/Critical  247 non-null    object 
 9   Tot Cases/1M      247 non-null    object 
 10  Deaths/1M         247 non-null    object 
 11  Total Tests       247 non-null    object 
 12  Tests/1M          247 non-null    object 
 13  Population        247 non-null    object 
 14  Continent         247 non-null    object 
dtypes: float64(3), object(12)
memory usage: 29.1+ KB


In [33]:
df=df.drop(columns=["New Cases", "Total Deaths", "New Deaths", "Total Recovered", "New Recovered", "Serious/Critical",
                "Deaths/1M", "Total Tests", "Tests/1M", "Continent"])
df = df.iloc[8:-8]
df.reset_index(inplace=True)
df=df.drop(columns=["index"])
df

,Country,Total Cases,Active Cases,Tot Cases/1M,Population
0,USA,109180951,906433,326103,334805269
1,India,45001268,0,31992,1406631776
2,France,40138560,0,612013,65584518
3,Germany,38553102,135709,459602,83883596
4,Brazil,37949944,993975,176222,215353593
...,...,...,...,...,...
226,Tokelau,80,80,58055,1378
227,Vatican City,29,0,36295,799
228,Western Sahara,10,0,16,626161
229,MS Zaandam,9,0,0,0


In [34]:
# Calculate the average number of active cases
average_active_cases = df['Active Cases'].astype(int).mean()

# Calculate the proportion of the total population that is currently infected
df['Population'] = df['Population'].astype(int)
df['Active Cases'] = df['Active Cases'].astype(int)
proportion_infected = df['Active Cases'].sum() / df['Population'].sum()
percentage_proportion_infected = proportion_infected *100

# Print the results
print(f"Average Active Cases: {average_active_cases}")
print(f"Proportion of Total Population Infected: {proportion_infected:.8f}")
print(f"Percentage of proportion of Total Population Infected: {percentage_proportion_infected:.3f}%")

Average Active Cases: 32473.79220779221
Proportion of Total Population Infected: 0.00094418
Percentage of proportion of Total Population Infected: 0.094%
